Data Preprocessing

In [1]:
#Importing the libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.manifold import TSNE
import umap

In [2]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#File path
df = pd.read_csv("/content/drive/MyDrive/DSPL/traincsv.csv")  # This line reads the CSV file into a DataFrame named 'df'

<ipython-input-3-16776ae4abf0>:2: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/DSPL/traincsv.csv")  # This line reads the CSV file into a DataFrame named 'df'


In [4]:
# Data Understanding
print("Initial Data Info:")
print(df.info())

Initial Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774155 entries, 0 to 774154
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Customer_ID       774153 non-null  float64
 1   outlet_city       774153 non-null  object 
 2   luxury_sales      774120 non-null  object 
 3   fresh_sales       774114 non-null  object 
 4   dry_sales         774125 non-null  object 
 5   cluster_catgeory  774154 non-null  object 
dtypes: float64(1), object(5)
memory usage: 35.4+ MB
None


In [5]:
#Describe
df.describe(include='all')

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales,cluster_catgeory
count,7.741530e+05,774153,774120,774114,774125.00,774154.0
unique,NaN,20,143897,257030,270131.00,18.0
top,NaN,Colombo,1029.6,6013.92,9168.72,1.0
freq,NaN,41031,62,28,23.00,188975.0
mean,1.038708e+07,NaN,NaN,NaN,NaN,NaN
std,2.234795e+05,NaN,NaN,NaN,NaN,NaN
min,1.000000e+07,NaN,NaN,NaN,NaN,NaN
25%,1.019354e+07,NaN,NaN,NaN,NaN,NaN
50%,1.038708e+07,NaN,NaN,NaN,NaN,NaN
75%,1.058062e+07,NaN,NaN,NaN,NaN,NaN


In [6]:
#Shape of the dataset
df.shape

(774155, 6)

In [7]:
#Check for duplicates
num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 0


In [8]:
# Removing Duplicates
df.drop_duplicates(inplace=True)

In [9]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values per Column:")
print(missing_values)

Missing Values per Column:
Customer_ID          2
outlet_city          2
luxury_sales        35
fresh_sales         41
dry_sales           30
cluster_catgeory     1
dtype: int64


In [10]:
# Remove rows with any missing values
df.dropna(inplace=True)

In [11]:
# Handle outliers using IQR
def remove_outliers(df, columns):
    for col in columns:
        if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
            Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
            df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

# Apply outlier removal to numeric columns
numeric_columns = ["luxury_sales", "fresh_sales", "dry_sales","cluster_category"]

In [12]:
# Convert relevant columns to float
columns_to_convert = ["luxury_sales", "fresh_sales", "dry_sales"]  # Adjust as needed
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

Data Transformation

In [13]:
# Convert cluster_category to numeric, forcing errors to NaN
df["cluster_catgeory"] = pd.to_numeric(df["cluster_catgeory"], errors="coerce")
valid_clusters = {1, 2, 3, 4, 5, 6}
df["cluster_catgeory"] = df["cluster_catgeory"].apply(lambda x: x if x in valid_clusters else None)
df["cluster_catgeory"].fillna(df["cluster_catgeory"].mode()[0], inplace=True)

<ipython-input-13-0ae4a99691d6>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["cluster_catgeory"].fillna(df["cluster_catgeory"].mode()[0], inplace=True)


In [14]:
# Normalization for train_df
scaler = MinMaxScaler()
sales_columns = ["luxury_sales", "fresh_sales", "dry_sales"]

# Replace 'train_cleaned' with 'df'
df[sales_columns] = scaler.fit_transform(df[sales_columns])
print(df)

        Customer_ID outlet_city  luxury_sales  fresh_sales  dry_sales  \
0        10493832.0    Kelaniya      0.109175     0.018966   0.354981   
1        10178643.0    Moratuwa      0.167720     0.041494   0.407956   
2        10513916.0     Wattala      0.275227     0.072681   0.641183   
3        10334589.0     Wattala      0.273082     0.091841   0.637052   
4        10458365.0    Kelaniya      0.283937     0.116281   0.647737   
...             ...         ...           ...          ...        ...   
774150   10197979.0     Gampaha      0.522090     0.251402   0.218414   
774151   10494575.0  Katunayake      0.860948     0.374724   0.427811   
774152   10565682.0     Colombo      0.711026     0.320062   0.308870   
774153   10351977.0       Kandy      0.894165     0.430568   0.403019   
774154   10637635.0     Negombo      0.820583     0.409541   0.452712   

        cluster_catgeory  
0                    4.0  
1                    1.0  
2                    4.0  
3              

In [15]:
# Encode the "outlet_city" column directly
label_encoder = LabelEncoder()
df["outlet_city"] = label_encoder.fit_transform(df["outlet_city"])

In [16]:
#Installing the preprocessed file
!pip install openpyxl
import openpyxl

df.to_excel('Trainpreprocessed_data.xlsx', index=False)